In [2]:
import os
import sys
import yaml
import rclpy
import numpy as np

# message libraries
from geometry_msgs.msg import PoseStamped, Pose

# moveit_py
from moveit.planning import MoveItPy
from moveit.core.robot_state import RobotState

# config file libraries
from moveit_configs_utils import MoveItConfigsBuilder
from ament_index_python.packages import get_package_share_directory

In [4]:
urdf_file = os.path.join(
    get_package_share_directory("niryo_ned_description"),
    "urdf/ned2",
    #"niryo_ned2.urdf.xacro",
    "niryo_ned2_gripper1_n_camera.urdf.xacro"

)
srdf_file = "/scripts/sim/niryo_ned2_gripper.srdf"

moveit_config = (
        MoveItConfigsBuilder("niryo_ned2")
        .robot_description(file_path=urdf_file)
        .joint_limits(file_path="config/joint_limits.yaml")
        .robot_description_semantic(file_path=srdf_file)
        .robot_description_kinematics(file_path="config/kinematics.yaml")
        .trajectory_execution(file_path="config/moveit_controllers.yaml")
        .planning_pipelines(
            pipelines=["ompl", "chomp", "pilz_industrial_motion_planner", "stomp"]
        )
        .to_moveit_configs()
    )

In [14]:

# 1) Exact URDF you load elsewhere
urdf_file = os.path.join(
    get_package_share_directory("niryo_ned_description"),
    "urdf", "ned2",
    "niryo_ned2_gripper1_n_camera.urdf.xacro",
)

# 2) Use the real MoveIt config package (adjust this if yours differs)
moveit_pkg = "niryo_ned2_moveit_config"  # <- ensure this package exists in your ROS env
pkg_share = get_package_share_directory(moveit_pkg)

srdf_file = "/scripts/sim/niryo_ned2_gripper.srdf"
kin_yaml  = os.path.join(pkg_share, "config", "kinematics.yaml")
jl_yaml   = os.path.join(pkg_share, "config", "joint_limits.yaml")
ctrl_yaml = os.path.join(pkg_share, "config", "moveit_controllers.yaml")
ompl_yaml = os.path.join(pkg_share, "config", "ompl_planning.yaml")

# 3) IMPORTANT: robot_name must match the URDF robot name
moveit_config = (
    MoveItConfigsBuilder(
        robot_name="niryo_ned2_gripper1_n_camera",
        package_name=moveit_pkg,  # lets relative paths resolve
    )
    .robot_description(file_path=urdf_file)               # xacro ok
    .robot_description_semantic(file_path=srdf_file)      # SRDF with matching robot name
    .robot_description_kinematics(file_path=kin_yaml)
    .joint_limits(file_path=jl_yaml)
    .trajectory_execution(file_path=ctrl_yaml)
    .planning_pipelines(pipelines=["ompl", "chomp", "pilz_industrial_motion_planner", "stomp"])               # enable OMPL
    .to_moveit_configs()
)

moveit = MoveItPy(node_name="moveit_py", config_dict=moveit_config.to_dict())
arm = moveit.get_planning_component("arm")

[INFO] [1763126951.891463363] [moveit_1573018812.moveit.py.cpp_initializer]: Initialize node parameters
[INFO] [1763126951.891509483] [moveit_1573018812.moveit.py.cpp_initializer]: Initialize node and executor
[WARN] [1763126951.892023789] [rcl.logging_rosout]: Publisher already registered for node name: 'moveit_py'. If this is due to multiple nodes with the same name then all logs for the logger named 'moveit_py' will go out over the existing publisher. As soon as any node with that name is destructed it will unregister the publisher, preventing any further logs for that name from being published on the rosout topic.
[INFO] [1763126951.900042612] [moveit_1573018812.moveit.py.cpp_initializer]: Spin separate thread
[INFO] [1763126951.903444649] [moveit_1573018812.moveit.ros.rdf_loader]: Loaded robot model in 0.00332695 seconds
[INFO] [1763126951.903470731] [moveit_1573018812.moveit.core.robot_model]: Loading robot model 'niryo_ned2_gripper1_n_camera'...
[INFO] [1763126951.942810424] [mo

RuntimeError: Failed to load planning pipelines from parameter server

In [15]:
# initialise rclpy (only for logging purposes)
#rclpy.init()

# instantiate moveit_py instance and a planning component for the panda_arm
ned2 = MoveItPy(node_name="moveit_py", config_dict=moveit_config.to_dict())
ned2 = panda.get_planning_component("ned2_arm")

[INFO] [1763126960.774079115] [moveit_1573018812.moveit.py.cpp_initializer]: Initialize node parameters
[INFO] [1763126960.774129635] [moveit_1573018812.moveit.py.cpp_initializer]: Initialize node and executor
[WARN] [1763126960.774877789] [rcl.logging_rosout]: Publisher already registered for node name: 'moveit_py'. If this is due to multiple nodes with the same name then all logs for the logger named 'moveit_py' will go out over the existing publisher. As soon as any node with that name is destructed it will unregister the publisher, preventing any further logs for that name from being published on the rosout topic.
[INFO] [1763126960.791277089] [moveit_1573018812.moveit.py.cpp_initializer]: Spin separate thread
[INFO] [1763126960.796381819] [moveit_1573018812.moveit.ros.rdf_loader]: Loaded robot model in 0.00495614 seconds
[INFO] [1763126960.796401405] [moveit_1573018812.moveit.core.robot_model]: Loading robot model 'niryo_ned2_gripper1_n_camera'...
[INFO] [1763126960.824674209] [mo

RuntimeError: Failed to load planning pipelines from parameter server

In [ ]:
def plan_and_execute(
        robot,
        planning_component,
        single_plan_parameters=None,
        multi_plan_parameters=None,
):
        """A helper function to plan and execute a motion."""
        # plan to goal
        if multi_plan_parameters is not None:
                plan_result = planning_component.plan(
                        multi_plan_parameters=multi_plan_parameters
                )
        elif single_plan_parameters is not None:
                plan_result = planning_component.plan(
                single_plan_parameters=single_plan_parameters
        )
        else:
                plan_result = planning_component.plan()

        # execute the plan
        if plan_result:
                robot_trajectory = plan_result.trajectory
                robot.execute(robot_trajectory, controllers=[])
        else:
                print("Planning failed")